In [1]:
# TODO make the same thing with classes karmedexperiment and karmedcompetition
# TODO implement proper graphs like in the former version 
# rl-specialization-coursera.exercises
# TODO think of implementing multicore version
# runtimes for 1e6 iterations measured 20250206, 9 am
# @ThinkPad: 25.4s
# @ThinkPad old approach with 2 cores: 27.5s
# @ThinkPad old approach with 6 cores: 11.4s (only p-cores?)
# @ThinkPad old approach with 16 cores: 8.9s (p- and e-cores)

# training a basic agent

import gymnasium as gym
import matplotlib.pyplot as plt
import numpy as np

from tqdm import tqdm
import karmedbandit
from karmedagents import BanditAgent

episodes = 1_00
timesteps = 1_00

env = gym.make("BanditEnv-v0")
# env.random_seed = 42

obs, info = env.reset()
print(f'Initial bandit arms q-values:{env.get_wrapper_attr("arms_q_values")}')

agent = BanditAgent(env=env, policy="random")

# TODO implement Metrics
# plt.title(f'Average loss per timestep over {runs} runs\n{title}')
# plt.title(f'Average rewards per timestep over {runs} runs\n{title}')
# plt.title(f'Cumulative Average rewards per timestep over {runs} runs\n{title}')
# plt.title(f'Percentage optimal actions per timestep over {runs} runs\n{title}')

for episode in tqdm(range(episodes)):
    obs, info = env.reset()

    for step in range(timesteps):  # Perform some steps
        action = agent.get_action() #Example
        obs, reward, terminated, truncated, info = env.step(action)
        agent.update(reward)
        step_error, optimal_action = info.get('step_error'), info.get('optimal_action')
        # print (step_error, optimal_action)

print(f'Final bandit arms q-values:{env.get_wrapper_attr("arms_q_values")}')

# env.close()

-------------------------
Bandit configuration
-------------------------
bandit_actions: 10
true_q_value_mean: 0.5
true_q_value_std: 0.0
q_value_std: 0.1
q_drift_mean: 0.0
q_drift_std: 0.0
random_seed: None
-------------------------
Initial bandit arms q-values:[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]


100%|██████████| 100/100 [00:00<00:00, 343.97it/s]

Final bandit arms q-values:[0.36936227 0.56821102 0.58782945 0.61990178 0.43582268 0.47077711
 0.6237416  0.49539517 0.3764956  0.58312161]


In [ ]:
# testing whether the env instantiation and use works in .ipynb

from karmedbandit import BanditEnv
env = BanditEnv()

# XXX info: gymnasium env self test. Run as required.
# The output in the terminal will indicate whether tests passed or failed.
# check_env(env = env)

print(env.true_q_value_mean)
obs, info = env.reset()
print("Initial arms_q_values:", env.arms_true_q_values)
print ("#"*79)
for _ in range(10_000):  # Perform some steps
    action = env.action_space.sample() #Example
    obs, reward, terminated, truncated, info = env.step(action)

print("Updated arms_q_values:", np.round(env.arms_q_values, 3))

env.close()

In [ ]:
#testing the arms_true_q_values creation works properly
import numpy as np
bandit_actions = 10

bandit_actions: int = 10
true_q_value_mean = 0.5, # 
true_q_value_std = 0., # for a uniform initial q-value set this to zero

arms_true_q_values = np.random.normal(true_q_value_mean, true_q_value_std, bandit_actions)

print(arms_true_q_values)

In [ ]:
# testing the ingest of BanditParams into BanditEnv

import inspect

import gymnasium as gym

class BanditParams:
    arms: int = 10
    qdrift_mean: float = 0.0
    qdrift_std: float = 0.0
    stationary: bool = True

class BanditEnv(gym.Env, BanditParams):

    metadata = {"render_modes": ["human"], "render_fps": 4}
    
    def __init__(self):
        super().__init__()
        pass

env = BanditEnv()
# print(env.arms)
# env.arms = 6
# print(env.arms)

print(*[f"{key}: {value}" for key, value in env.__dict__.items()], sep="\n")

In [ ]:
# Testing a violin chart to represent a distribution

'''Figure 2.1: An example bandit problem from the 10-armed testbed. The true value q⇤(a) of
each of the ten actions was selected according to a normal distribution with mean zero and unit
variance, and then the actual rewards were selected according to a mean q⇤(a), unit-variance
normal distribution, as suggested by these gray distributions.'''

import matplotlib.pyplot as plt
import numpy as np

# Generate the data
arms = 10
arms_q_values = np.random.normal(0, 1, arms)
timesteps = 10000
arms_values_over_time = np.zeros(timesteps)

arm = np.min(arms_q_values)
for i in range(timesteps):
    arms_values_over_time[i] = np.random.normal(arm, 1)

# Create a violin plot based on the data in arms_values_over_time
plt.violinplot(arms_values_over_time)
plt.title('Violin Plot of Arms Values Over Time')
plt.xlabel('Arms')
plt.ylabel('Values')

# Calculate the mean of the distribution
mean_value = np.mean(arms_values_over_time)

# Add a shorter marker/error line for the mean of the distribution
plt.axhline(mean_value, color='r', linestyle='--', linewidth=1, xmin=0.45, xmax=0.55)

# Get the current axis
ax = plt.gca()

# Get the right end of the error line
xmax = ax.get_xlim()[1] * 0.55
print (xmax, ax.get_xlim()[1])

# Add text relative to the right end of the error line
plt.text(1.45 * xmax + 0.02 * ax.get_xlim()[1], mean_value, f'q({np.argmin(arms_q_values)})', color='r', verticalalignment='center')

# Show the plot
plt.show()